## Pre-requisites

In [1]:
# Import libraries
import os
import sys
import shutil

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, "..")) # absolute path for ekr_rag directory
repo_dir = os.path.abspath(os.path.join(kit_dir, "..")) # absolute path for starter-kit directory
print('kit_dir: %s'%kit_dir)
print('repo_dir: %s'%repo_dir)

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.document_retrieval import DocumentRetrieval
from utils.parsing.sambaparse import SambaParse, parse_doc_universal

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')
PERSIST_DIRECTORY = os.path.join(kit_dir,f"data/my-vector-db")

kit_dir: /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever
repo_dir: /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit


/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document loading and splitting

In [2]:
# Specify PDF folder location
pdf_folder = kit_dir+'/data/test'

# Initialize DocumentRetrieval class
documentRetrieval =  DocumentRetrieval()

# Parse and chunk the documents
additional_metadata = {}
_, _, text_chunks = parse_doc_universal(doc=pdf_folder, additional_metadata=additional_metadata)
print('Nb of chunks: %d'%len(text_chunks))


2024-07-14 16:53:32,732 [INFO] - Deleting contents of output directory: ./output
2024-07-14 16:53:32,760 [INFO] - Running command: unstructured-ingest local --output-dir ./output --num-processes 2 --strategy auto --ocr-languages eng --encoding utf-8 --fields-include element_id,text,type,metadata,embeddings --metadata-exclude  --metadata-include  --pdf-infer-table-structure --input-path "/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/data/test" --recursive --verbose --chunking-strategy by_title --chunk-max-characters 1500 --chunk-overlap 300 --chunk-combine-text-under-n-chars 1500 --chunk-elements
2024-07-14 16:53:32,763 [INFO] - This may take some time depending on the size of your data. Please be patient...
/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/dataclasses_json/core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type 

Nb of chunks: 10


## Vectorization and storage

In [3]:
# Create vector store
embeddings = documentRetrieval.load_embedding_model()
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"The directory Chroma has been deleted.")
#vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=None)
vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=PERSIST_DIRECTORY)

/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-07-14 16:54:31,931 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-07-14 16:54:35,989 [INFO] - Use pytorch device: cpu


max_seq_length  512
The directory Chroma has been deleted.


2024-07-14 16:54:38,822 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-14 16:54:51,207 [INFO] - Vector store saved to /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/data/my-vector-db


## Retrieval and generation

In [4]:
# Create conversation chain
documentRetrieval.init_retriever(vectorstore)
conversation = documentRetrieval.get_qa_retrieval_chain()

In [5]:
# Ask questions about your data
user_question = "What is a composition of experts?"

response = conversation.invoke({"question":user_question})
print(response['question'])
print(response['answer'])

for i in range(0,len(response['source_documents'])):
    print('\nSource #%d:'%(i+1))
    print(response['source_documents'][i].page_content)
    print(response['source_documents'][i].metadata)

What is a composition of experts?
A composition of experts (CoE) is a modular approach that lowers the cost and complexity of training and serving AI models. It involves combining many smaller expert models, each with several orders of magnitude fewer parameters, to match or exceed the capabilities of monolithic large language models (LLMs).

Source #1:
SambaNova Systems, Inc. ﬁrst.last@sambanova.ai

like GPT-4 have paved the way for modern generative AI serving, and maintaining monolithic applications. Training, LLMs at remains prohibitively expensive and challenging. The disproportionate increase in compute- to-memory ratio of modern AI accelerators have created a memory wall, necessitating new methods to deploy AI. Recent research has shown that a composition of many smaller expert models, each with several orders of magnitude fewer parameters, can match or exceed the capabilities of monolithic LLMs. Composition of Experts (CoE) is a modular approach that lowers the cost and complex